In [1]:
import pandas as pd

path = r"C:\Users\Avijit\Desktop\Synthetic KYC and Transaction Risk Dataset\clients_with_fatf_ofac.csv"
df = pd.read_csv(path)

df.head()


,client_id,client_name,client_type,sector,sector_risk,country,pep_flag,sanctions_flag,fatf_country_flag,ofac_country_flag,sectoral_sanctions_flag,ownership_opacity_score
0,1,Wells-Turner,Financial Institution,NGO/Charity,High,JP,0,0,0,0,0,0.0
1,2,"Goodman, Hendrix and Garcia",Financial Institution,Import/Export,Medium,CH,0,0,0,0,0,0.0
2,3,Phillips-Hanson,NGO,NGO/Charity,High,AE,0,1,0,0,0,0.5
3,4,"Kim, Anderson and Espinoza",Financial Institution,Defense/Arms,High,RU,1,0,0,1,1,0.0
4,5,Marks-Thompson,Financial Institution,Financial Services,Medium,AU,0,0,0,0,1,0.0


In [2]:
df.info()
df.describe(include="all")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   client_id                2000 non-null   int64  
 1   client_name              2000 non-null   object 
 2   client_type              2000 non-null   object 
 3   sector                   2000 non-null   object 
 4   sector_risk              2000 non-null   object 
 5   country                  2000 non-null   object 
 6   pep_flag                 2000 non-null   int64  
 7   sanctions_flag           2000 non-null   int64  
 8   fatf_country_flag        2000 non-null   int64  
 9   ofac_country_flag        2000 non-null   int64  
 10  sectoral_sanctions_flag  2000 non-null   int64  
 11  ownership_opacity_score  2000 non-null   float64
dtypes: float64(1), int64(6), object(5)
memory usage: 187.6+ KB


,client_id,client_name,client_type,sector,sector_risk,country,pep_flag,sanctions_flag,fatf_country_flag,ofac_country_flag,sectoral_sanctions_flag,ownership_opacity_score
count,2000.000000,2000,2000,2000,2000,2000,2000.000000,2000.000000,2000.000000,2000.00000,2000.00000,2000.000000
unique,NaN,1899,4,11,3,21,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Johnson Inc,NGO,Import/Export,High,CH,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,7,512,200,920,113,NaN,NaN,NaN,NaN,NaN,NaN
mean,1000.500000,NaN,NaN,NaN,NaN,NaN,0.057000,0.027500,0.089500,0.30050,0.28750,0.111250
std,577.494589,NaN,NaN,NaN,NaN,NaN,0.231901,0.163576,0.285535,0.45859,0.45271,0.270712
min,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
25%,500.750000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
50%,1000.500000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
75%,1500.250000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.00000,1.00000,0.000000


In [3]:
def compute_customer_risk_score(row):
    score = 0

    # 1. Sanctions risk (highest priority)
    if row["sanctions_flag"] == 1:
        score += 100

    if row["ofac_country_flag"] == 1:
        score += 80

    if row["sectoral_sanctions_flag"] == 1:
        score += 60

    # 2. Geographic risk
    if row["fatf_country_flag"] == 1:
        score += 40

    # 3. PEP risk
    if row["pep_flag"] == 1:
        score += 30

    # 4. Sector risk
    if row["sector_risk"] == "High":
        score += 25
    elif row["sector_risk"] == "Medium":
        score += 15

    # 5. Ownership opacity
    if row["ownership_opacity_score"] > 0.7:
        score += 20
    elif row["ownership_opacity_score"] >= 0.3:
        score += 10

    return score


In [4]:
df["customer_risk_score"] = df.apply(compute_customer_risk_score, axis=1)


In [5]:
df[["customer_risk_score"]].describe()


,customer_risk_score
count,2000.000000
mean,67.442500
std,57.640549
min,0.000000
25%,15.000000
50%,75.000000
75%,105.000000
max,305.000000


In [6]:
df[df["sanctions_flag"] == 1][
    ["client_name", "customer_risk_score"]
].head()


,client_name,customer_risk_score
2,Phillips-Hanson,135
93,"Lewis, Welch and Charles",100
96,Powell-Johnson,285
163,Beltran and Sons,110
253,Saunders-Contreras,305


In [8]:
def customer_risk_category(score):
    if score <= 30:
        return "Low"
    elif score <= 70:
        return "Medium"
    else:
        return "High"

df["customer_risk_category"] = df["customer_risk_score"].apply(customer_risk_category)


In [9]:
df["customer_risk_category"].value_counts(normalize=True) * 100


customer_risk_category
High      51.80
Low       43.15
Medium     5.05
Name: proportion, dtype: float64

In [10]:
def identify_risk_drivers(row):
    drivers = []

    if row["sanctions_flag"] == 1:
        drivers.append("Sanctions")

    if row["ofac_country_flag"] == 1:
        drivers.append("OFAC Country")

    if row["fatf_country_flag"] == 1:
        drivers.append("FATF High-Risk Country")

    if row["pep_flag"] == 1:
        drivers.append("PEP")

    if row["sector_risk"] == "High":
        drivers.append("High-Risk Sector")
    elif row["sector_risk"] == "Medium":
        drivers.append("Medium-Risk Sector")

    if row["ownership_opacity_score"] > 0.7:
        drivers.append("Opaque Ownership")
    elif row["ownership_opacity_score"] >= 0.3:
        drivers.append("Moderate Ownership Opacity")

    return drivers


In [11]:
df["risk_drivers"] = df.apply(identify_risk_drivers, axis=1)


In [12]:
df.loc[df["customer_risk_category"] == "High",
       ["client_name", "customer_risk_score", "risk_drivers"]
].head()


,client_name,customer_risk_score,risk_drivers
2,Phillips-Hanson,135,"[Sanctions, High-Risk Sector, Moderate Ownersh..."
3,"Kim, Anderson and Espinoza",195,"[OFAC Country, PEP, High-Risk Sector]"
4,Marks-Thompson,75,[Medium-Risk Sector]
6,Lambert-Ruiz,85,[High-Risk Sector]
8,Vega-Randall,95,"[Medium-Risk Sector, Opaque Ownership]"


In [13]:
df.to_csv("clients_scored_customer_risk.csv", index=False)


In [14]:
cust = pd.read_csv("clients_scored_customer_risk.csv")


In [15]:
cust.head()

,client_id,client_name,client_type,sector,sector_risk,country,pep_flag,sanctions_flag,fatf_country_flag,ofac_country_flag,sectoral_sanctions_flag,ownership_opacity_score,customer_risk_score,customer_risk_category,risk_drivers
0,1,Wells-Turner,Financial Institution,NGO/Charity,High,JP,0,0,0,0,0,0.0,25,Low,['High-Risk Sector']
1,2,"Goodman, Hendrix and Garcia",Financial Institution,Import/Export,Medium,CH,0,0,0,0,0,0.0,15,Low,['Medium-Risk Sector']
2,3,Phillips-Hanson,NGO,NGO/Charity,High,AE,0,1,0,0,0,0.5,135,High,"['Sanctions', 'High-Risk Sector', 'Moderate Ow..."
3,4,"Kim, Anderson and Espinoza",Financial Institution,Defense/Arms,High,RU,1,0,0,1,1,0.0,195,High,"['OFAC Country', 'PEP', 'High-Risk Sector']"
4,5,Marks-Thompson,Financial Institution,Financial Services,Medium,AU,0,0,0,0,1,0.0,75,High,['Medium-Risk Sector']


In [16]:
path = r"C:\Users\Avijit\Desktop\Synthetic KYC and Transaction Risk Dataset\transactions_with_fatf_ofac.csv"
tx = pd.read_csv(path)

tx.head()

,transaction_id,client_id,amount,transaction_type,timestamp,client_country,counterparty_country,ofac_match_flag,fatf_country_flag,structuring_pattern_flag,rapid_movement_flag,trade_mispricing_flag
0,1,1412,1027.64,Check,2025-08-20 09:21:56,IN,SG,0,0,0,0,0
1,2,1035,518.82,SWIFT,2025-09-07 07:52:58,RU,NL,0,0,0,0,0
2,3,704,460.78,SWIFT,2025-09-28 03:00:43,LB,IN,0,0,0,0,0
3,4,1073,1625.76,Wire,2025-08-22 04:57:11,AE,CA,0,0,0,0,0
4,5,1622,1856.47,SWIFT,2025-08-07 19:49:50,CH,IN,0,0,0,0,0


In [17]:
tx.info()
tx.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   transaction_id            50000 non-null  int64  
 1   client_id                 50000 non-null  int64  
 2   amount                    50000 non-null  float64
 3   transaction_type          50000 non-null  object 
 4   timestamp                 50000 non-null  object 
 5   client_country            50000 non-null  object 
 6   counterparty_country      50000 non-null  object 
 7   ofac_match_flag           50000 non-null  int64  
 8   fatf_country_flag         50000 non-null  int64  
 9   structuring_pattern_flag  50000 non-null  int64  
 10  rapid_movement_flag       50000 non-null  int64  
 11  trade_mispricing_flag     50000 non-null  int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 4.6+ MB


,transaction_id,client_id,amount,ofac_match_flag,fatf_country_flag,structuring_pattern_flag,rapid_movement_flag,trade_mispricing_flag
count,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,25000.500000,1001.97282,4196.137859,0.035800,0.012040,0.006940,0.049200,0.000200
std,14433.901067,577.71469,10975.575681,0.185793,0.109065,0.083018,0.216288,0.014141
min,1.000000,1.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12500.750000,499.00000,639.327500,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25000.500000,1006.00000,1582.185000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37500.250000,1502.25000,3413.925000,0.000000,0.000000,0.000000,0.000000,0.000000
max,50000.000000,2000.00000,324732.560000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
TX_WEIGHTS = {
    "ofac_match_flag": 100,
    "fatf_country_flag": 40,
    "structuring_pattern_flag": 30,
    "rapid_movement_flag": 25,
    "trade_mispricing_flag": 35
}

In [19]:
def compute_transaction_risk(row):
    score = 0

    if row["ofac_match_flag"] == 1:
        score += 100

    if row["fatf_country_flag"] == 1:
        score += 40

    if row["structuring_pattern_flag"] == 1:
        score += 30

    if row["rapid_movement_flag"] == 1:
        score += 25

    if row["trade_mispricing_flag"] == 1:
        score += 35

    return score

In [20]:
tx["transaction_risk_score"] = tx.apply(compute_transaction_risk, axis=1)

In [21]:
tx["transaction_risk_score"].describe()

count    50000.000000
mean         5.506800
std         22.218281
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        195.000000
Name: transaction_risk_score, dtype: float64

In [22]:
tx[tx["ofac_match_flag"] == 1][
    ["transaction_id", "transaction_risk_score"]
].head()

,transaction_id,transaction_risk_score
25,26,140
36,37,100
98,99,100
101,102,100
106,107,140


In [23]:
def transaction_alert(score):
    if score >= 75:
        return 1   # Alert
    return 0

In [24]:
tx["transaction_alert_flag"] = tx["transaction_risk_score"].apply(transaction_alert)

In [25]:
tx["transaction_alert_flag"].value_counts(normalize=True) * 100

transaction_alert_flag
0    96.42
1     3.58
Name: proportion, dtype: float64

In [26]:
features = [
    "amount",
    "fatf_country_flag",
    "ofac_match_flag",
    "structuring_pattern_flag",
    "rapid_movement_flag",
    "trade_mispricing_flag"
]

In [27]:
from sklearn.preprocessing import StandardScaler

X = tx[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [28]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(
    n_estimators=100,
    contamination=0.02,   # 2% anomalies
    random_state=42
)

tx["anomaly_flag"] = iso.fit_predict(X_scaled)

In [29]:
tx["anomaly_flag"] = tx["anomaly_flag"].map({1: 0, -1: 1})

In [30]:
tx["anomaly_flag"].value_counts(normalize=True) * 100

anomaly_flag
0    98.036
1     1.964
Name: proportion, dtype: float64

In [31]:
tx["final_transaction_risk_score"] = (
    tx["transaction_risk_score"] +
    (tx["anomaly_flag"] * 50)
)

In [32]:
tx["final_transaction_alert"] = tx["final_transaction_risk_score"].apply(
    lambda x: 1 if x >= 75 else 0
)

In [33]:
tx["final_transaction_alert"].value_counts(normalize=True) * 100

final_transaction_alert
0    96.0
1     4.0
Name: proportion, dtype: float64

In [34]:
final_df = tx.merge(
    cust[[
        "client_id",
        "customer_risk_score",
        "customer_risk_category"
    ]],
    on="client_id",
    how="left"
)


In [37]:
final_df["final_aml_risk_score"] = (
    0.6 * final_df["customer_risk_score"] +
    0.4 * final_df["final_transaction_risk_score"]
)

In [38]:
final_df["final_aml_alert"] = final_df["final_aml_risk_score"].apply(
    lambda x: 1 if x >= 80 else 0
)

In [39]:
final_df["final_aml_alert"].value_counts(normalize=True) * 100

final_aml_alert
0    83.942
1    16.058
Name: proportion, dtype: float64

In [40]:
final_df["final_aml_risk_score"].describe()

count    50000.000000
mean        43.086220
std         38.084634
min          0.000000
25%         12.000000
50%         45.000000
75%         63.000000
max        259.000000
Name: final_aml_risk_score, dtype: float64

In [41]:
final_df["final_aml_risk_score"].quantile([0.90, 0.92, 0.95, 0.97, 0.99])

0.90     99.0
0.92     99.0
0.95    117.0
0.97    123.0
0.99    163.0
Name: final_aml_risk_score, dtype: float64

In [42]:
final_df["final_aml_alert"] = final_df["final_aml_risk_score"].apply(
    lambda x: 1 if x >= 117 else 0
)

In [43]:
final_df["final_aml_alert"].value_counts(normalize=True) * 100

final_aml_alert
0    94.856
1     5.144
Name: proportion, dtype: float64

In [44]:
final_df.head()

,transaction_id,client_id,amount,transaction_type,timestamp,client_country,counterparty_country,ofac_match_flag,fatf_country_flag,structuring_pattern_flag,...,trade_mispricing_flag,transaction_risk_score,transaction_alert_flag,anomaly_flag,final_transaction_risk_score,final_transaction_alert,customer_risk_score,customer_risk_category,final_aml_risk_score,final_aml_alert
0,1,1412,1027.64,Check,2025-08-20 09:21:56,IN,SG,0,0,0,...,0,0,0,0,0,0,25,Low,15.0,0
1,2,1035,518.82,SWIFT,2025-09-07 07:52:58,RU,NL,0,0,0,...,0,0,0,0,0,0,80,High,48.0,0
2,3,704,460.78,SWIFT,2025-09-28 03:00:43,LB,IN,0,0,0,...,0,0,0,0,0,0,0,Low,0.0,0
3,4,1073,1625.76,Wire,2025-08-22 04:57:11,AE,CA,0,0,0,...,0,0,0,0,0,0,65,Medium,39.0,0
4,5,1622,1856.47,SWIFT,2025-08-07 19:49:50,CH,IN,0,0,0,...,0,0,0,0,0,0,15,Low,9.0,0


In [45]:
final_aml_output = final_df[
    [
        "transaction_id",
        "client_id",
        "customer_risk_score",
        "final_transaction_risk_score",
        "final_aml_risk_score",
        "final_aml_alert"
    ]
]

In [46]:
final_aml_output.head()

,transaction_id,client_id,customer_risk_score,final_transaction_risk_score,final_aml_risk_score,final_aml_alert
0,1,1412,25,0,15.0,0
1,2,1035,80,0,48.0,0
2,3,704,0,0,0.0,0
3,4,1073,65,0,39.0,0
4,5,1622,15,0,9.0,0


In [47]:
final_aml_output.to_csv("final_aml_alerts.csv", index=False)